# SQLite Exploration of Virginia's Adjusted Census files

##### Imports

In [1]:
import pandas as pd

# Always display all columns (no matter how many there are)
pd.set_option('display.max_columns', None)

from plotnine import *

/Users/sm/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# Necessary imports
import pandas as pd
from plotnine import *

# I don't think we use these?
from mizani.breaks import date_breaks
from mizani.formatters import date_format

In [3]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [4]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [30]:
%%R

# My commonly used R imports

require('ggplot2')
require('readr')
require('extrafont')
require('dplyr')
require('reshape2')
require('explore')
require('tidyr')
require('forcats')
require('knitr')
require('ragg')
require('palmerpenguins')
# require('theme538')

R[write to console]: Loading required package: ragg



In [6]:
df = pd.DataFrame(
    np.random.randint(0,100,size=(100, 2)), columns=list('AB'))
df.head()

,A,B
0,95,17
1,96,64
2,23,40
3,11,54
4,26,62


In [7]:
# %%R -i df

# # EXAMPLE R CELL
# df
# plt <- ggplot(df) +
#     geom_point(aes(A,B))

# plt

----

##### U.S. Census data Abbreviations

Abbreviations Glossary https://www.census.gov/pred/www/rpts/TXE%20Program%20Summary%20Appendix%20B.pdf

More codes http://starr.tamu.edu/files/2013/01/Census-Codes.pdf

FILEID = file id

STUSAB = State abbreviation

SUMLEV = Census Bureau code to indicate geography type. 
For example, in the raw census data, all states have a SUMLEV code of “040.”

CHARITER = characteristic iteration

CIFSN = characteristic iteration file sequence number 

LOGRECNO = unique logical record number 


P0010001 = Total population Total

P0020001 = Total population Total

P0030001 = Total population 18 years and over Total

P0050001 = Total Group Quarters Population

P0050003 = Correctional facilities for adults

P0050004 = Juvenile facilities

In [8]:
# Census GEOCODE = 60014001001000 is the same as adjusted BLOCK20 = 60014001001000
# 6 = state , 001 = county, 400100 = tract, 1000 = block

-----

## Virginia

##### 2020 Redistricting Data for Virginia at the Block level (Adjusted)

From Data Hub ReadMe file: This dataset was retrieved from the Virginia Redistricting Commission at https://virginiaredistricting.org/PageReader.aspx?page=2020DataDownload 09/14/2021 

https://redistrictingdatahub.org/dataset/virginia-block-county-and-vtd-pl-94-171-2020-official/

##Additional Notes
The Virginia Redistricting Commisions notes: "For prisoner adjustments, two fields were added:
PRADJPOPD: The number of prisoners added or removed from the geographic features total population (TAPERSONS)
ADJPOP: The adjusted population
The field containing the total population from the Census Bureau (TAPERSONS) was not modified."


##### Using sqlite to understand the VA adjusted redistricting data file, locating the relevant adjusted census data

In [9]:
#"..//data//source//RDH-data//va_pl2020_official_gpkg//statewide20.gpkg"

In [14]:
import sqlite3
conn = sqlite3.connect('..//data//source//RDH-data//va_pl2020_official_gpkg//statewide20.gpkg')                                
cursor = conn.cursor()

In [15]:
print(conn.total_changes)

0


Identifying tables within the database file

In [16]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('gpkg_spatial_ref_sys',), ('gpkg_contents',), ('gpkg_ogr_contents',), ('gpkg_geometry_columns',), ('gpkg_tile_matrix_set',), ('gpkg_tile_matrix',), ('sqlite_sequence',), ('gpkg_extensions',), ('gpkg_metadata',), ('gpkg_metadata_reference',), ('rtree_counties_geom_rowid',), ('rtree_counties_geom_node',), ('rtree_counties_geom_parent',), ('rtree_vtds_geom_rowid',), ('rtree_vtds_geom_node',), ('rtree_vtds_geom_parent',), ('rtree_blocks_geom_rowid',), ('rtree_blocks_geom_node',), ('rtree_blocks_geom_parent',), ('rtree_counties_geom',), ('rtree_vtds_geom',), ('rtree_blocks_geom',), ('blocks',), ('counties',), ('vtds',)]


Identifying contents of different tables

In [17]:
data=cursor.execute('''SELECT * FROM gpkg_contents''')
print(data.description)

(('table_name', None, None, None, None, None, None), ('data_type', None, None, None, None, None, None), ('identifier', None, None, None, None, None, None), ('description', None, None, None, None, None, None), ('last_change', None, None, None, None, None, None), ('min_x', None, None, None, None, None, None), ('min_y', None, None, None, None, None, None), ('max_x', None, None, None, None, None, None), ('max_y', None, None, None, None, None, None), ('srs_id', None, None, None, None, None, None))


In [18]:
data2=cursor.execute('''SELECT * FROM gpkg_ogr_contents''')
print(data2.description)

(('table_name', None, None, None, None, None, None), ('feature_count', None, None, None, None, None, None))


In [31]:
data2=cursor.execute('''SELECT * FROM blocks''')
# print(data2.description)

In [20]:
# data3=cursor.execute('''SELECT * FROM counties''')
# print(data3.description)

Getting more detailed information on the blocks table — Displaying all the columns

In [21]:
# Import module
import sqlite3

# Connecting to sqlite
conn = sqlite3.connect('..//data//source//RDH-data//va_pl2020_official_gpkg//statewide20.gpkg')

# Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [22]:
# Display columns
print('\nColumns in blocks table:')
data=cursor.execute('''SELECT * FROM blocks''')
for column in data.description:
    print(column[0])


Columns in blocks table:
fid
geom
STATEFP20
COUNTYFP20
TRACTCE20
BLOCKCE20
GEOID20
NAME20
MTFCC20
UR20
UACE20
UATYPE20
FUNCSTAT20
ALAND20
AWATER20
INTPTLAT20
INTPTLON20
FILEID
STUSAB
SUMLEV
GEOVAR
GEOCOMP
CHARITER
CIFSN
LOGRECNO
GEOID
GEOCODE
REGION
DIVISION
STATE
STATENS
COUNTY
COUNTYCC
COUNTYNS
COUSUB
COUSUBCC
COUSUBNS
SUBMCD
SUBMCDCC
SUBMCDNS
ESTATE
ESTATECC
ESTATENS
CONCIT
CONCITCC
CONCITNS
PLACE
PLACECC
PLACENS
TRACT
BLKGRP
BLOCK
AIANHH
AIHHTLI
AIANHHFP
AIANHHCC
AIANHHNS
AITS
AITSFP
AITSCC
AITSNS
TTRACT
TBLKGRP
ANRC
ANRCCC
ANRCNS
CBSA
MEMI
CSA
METDIV
NECTA
NMEMI
CNECTA
NECTADIV
CBSAPCI
NECTAPCI
UA
UATYPE
UR
CD116
CD118
CD119
CD120
CD121
SLDU18
SLDU22
SLDU24
SLDU26
SLDU28
SLDL18
SLDL22
SLDL24
SLDL26
SLDL28
VTD
VTDI
ZCTA
SDELM
SDSEC
SDUNI
PUMA
AREALAND
AREAWATR
BASENAME
NAME
FUNCSTAT
GCUNI
POP100
HU100
INTPTLAT
INTPTLON
LSADC
PARTFLAG
UGA
TAPERSONS
TA1RACE
TAWHITEALN
TABLACKALN
TAAMINDALN
TAASIANALN
TANHPOALN
TAOTHERALN
TA2RACE
TAPOP2RACE
TAWHTBLACK
TAWHAIAN
TAWHTASIAN
TAWHNHPO
TAW

##### Used SQLite Studio for export of table into csv 

Used sqlite tutorial https://www.sqlitetutorial.net/sqlite-tutorial/sqlite-export-csv/ to export SQL database blocks table to a CSV file using SQlite Studio 

Reading in exported CSV to see if correct

In [35]:
df = pd.read_csv("..//data//processed//va_statewide20_converted.csv", encoding="utf-8")
df.head(3)

,fid,geom,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,FILEID,STUSAB,SUMLEV,GEOVAR,GEOCOMP,CHARITER,CIFSN,LOGRECNO,GEOID,GEOCODE,REGION,DIVISION,STATE,STATENS,COUNTY,COUNTYCC,COUNTYNS,COUSUB,COUSUBCC,COUSUBNS,SUBMCD,SUBMCDCC,SUBMCDNS,ESTATE,ESTATECC,ESTATENS,CONCIT,CONCITCC,CONCITNS,PLACE,PLACECC,PLACENS,TRACT,BLKGRP,BLOCK,AIANHH,AIHHTLI,AIANHHFP,AIANHHCC,AIANHHNS,AITS,AITSFP,AITSCC,AITSNS,TTRACT,TBLKGRP,ANRC,ANRCCC,ANRCNS,CBSA,MEMI,CSA,METDIV,NECTA,NMEMI,CNECTA,NECTADIV,CBSAPCI,NECTAPCI,UA,UATYPE,UR,CD116,CD118,CD119,CD120,CD121,SLDU18,SLDU22,SLDU24,SLDU26,SLDU28,SLDL18,SLDL22,SLDL24,SLDL26,SLDL28,VTD,VTDI,ZCTA,SDELM,SDSEC,SDUNI,PUMA,AREALAND,AREAWATR,BASENAME,NAME,FUNCSTAT,GCUNI,POP100,HU100,INTPTLAT,INTPTLON,LSADC,PARTFLAG,UGA,TAPERSONS,TA1RACE,TAWHITEALN,TABLACKALN,TAAMINDALN,TAASIANALN,TANHPOALN,TAOTHERALN,TA2RACE,TAPOP2RACE,TAWHTBLACK,TAWHAIAN,TAWHTASIAN,TAWHNHPO,TAWHTOTHER,TABLAIAN,TABLKASIAN,TABLNHPO,TABLKOTHER,TAAIANAS,TAAIANNHP,TAAIANOR,TAASNHPO,TAASIANOTH,TANHPOOR,TA3RACES,TAWHBLAIN,TAWHBLAS,TAWHBLNHP,TAWHBLOR,TAWHAIANS,TAWHAIANHP,TAWHAIANOR,TAWASNHOPI,TAWHASOR,TAWHNHPOOT,TABLAIANS,TABLAIANHP,TABLAIANOR,TABLASNHOP,TABLASOR,TABLNHPOOR,TAAIANASHP,TAAIANASOR,TAAIANNHPO,TAASNHPOOR,TA4RACES,TAWHBLAINA,TAWHBLINHP,TAWHBLAINO,TAWHBLASHP,TAWHBLASOR,TAWHBLNHPO,TAWHAINSHP,TAWHAIANSO,TAWHAINHPO,TAWHASNHPO,TABLAINSHP,TABLAIANSO,TABLAINHPO,TABLASNHPO,TAAIANSHPO,TA5RACES,TAWHBINSHP,TAWHBLINSO,TAWHBINHPO,TAWHBLSHPO,TAWHINSHPO,TABLINSHPO,TA6RACES,TWHBINSHPO,THISPNHISP,TAHISPANIC,TNHISPANIC,TN1RACE,TNWHALN,TNBLKALN,TNAIANALN,TNASIANALN,TNNHPOALN,TNOTHRALN,TN2MRACES,TN2RACES,TNWHBL,TNWHAIAN,TNWHTASIAN,TNWHNHPOPI,TNWHOR,TNBLAIAN,TNBLASIAN,TNBLNHPOPI,TNBLOR,TNAIANAS,TNAIANNHPI,TNAIANOR,TNASNHPOPI,TNASOR,TNNHOPIOOR,TN3RACES,TNWHBLAIAN,TNWHBLAS,TNWHBLNHPI,TNWHBLOR,TNWHAIANAS,TNWHAIANHP,TNWHAIANOR,TNWHASNHPI,TNWHASOR,TNWHNHPOOR,TNBLAIANAS,TNBLAIANHP,TNBLAIANOR,TNBLASNHPI,TNBLASOR,TNBLNHPOOR,TNAIANASHP,TNAIANASOR,TNAIANNHPO,TNASNHPOOR,TN4RACES,TNWHBLAINS,TNWHBLINHP,TNWHBLAINO,TNWHBLASHP,TNWHBLASOR,TNWHBLNHPO,TNWHAINSHP,TNWHAIANSO,TNWHAINHPO,TNWHASNHPO,TNBLAINSHP,TNBLAIANSO,TNBLAINHPO,TNBLASNHPO,TNAIANSHPO,TN5RACES,TNWHBINSHP,TNWHBLINSO,TNWHBINHPO,TNWHBLSHPO,TNWHINSHPO,TNBLINSHPO,TN6RACES,TNWBINSHPO,VAPERSONS,VA1RACE,VAWHITEALN,VABLACKALN,VAAIANALN,VAASIANALN,VANHPOALN,VAOTHERALN,VA2MRACES,VA2RACES,VAWHBL,VAWHAIAN,VAWHAS,VAWHNHPO,VAWHOR,VABLAIAN,VABLASIAN,VABLNHOPI,VABLOR,VAAIANAS,VAAIANNHPI,VAAIANOR,VAASNHOPI,VAASOR,VANHPOOR,VA3RACES,VAWHBLAIAN,VAWHBLAS,VAWHBLNHPI,VAWHBLOR,VAWHAIANAS,VAWHAIANHP,VAWHAIANOR,VAWHASNHPI,VAWHASOR,VAWHNHPIOT,VABLAIANAS,VABLAIANHP,VABLAIANOR,VABLASNHPI,VABLASOR,VABLNHPIOT,VAAIANASHP,VAAIANASOR,VAAIANNHPO,VAASNHPOOR,VA4RACES,VAWHBLAINS,VAWHBLINHP,VAWHBLAINO,VAWHBLASHP,VAWHBLASOR,VAWHBLNHPO,VAWHAINSHP,VAWHAIANSO,VAWHAINHPO,VAWHASNHPO,VABLAINSHP,VABLAIANSO,VABLAINHPO,VABLASNHPO,VAAIANSHPO,VA5RACES,VAWHBINSHP,VAWHBLINSO,VAWHBINHPO,VAWHBLSHPO,VAWHINSHPO,VABLINSHPO,VA6RACES,VAWBINSHPO,VAHISNHISP,VAHISPANIC,VANHISPANC,VAN1RACE,VANWHTALN,VANBLKALN,VANAIANALN,VANASANALN,VANNHPOALN,VANORALN,VANM2RACES,VAN2RACES,VANWHBL,VANWHAIAN,VANWHAS,VANWHNHOPI,VANWHOR,VANBLAIAN,VANBLAS,VANBLNHOPI,VANBLOR,VANAIANAS,VANAIANNHP,VANAIANOR,VANASNHOPI,VANASOR,VANNHOPIOR,VAN3RACES,VANWHBLAIN,VANWHBLAS,VANWHBLNHP,VANWHBLOR,VANWHAIANS,VANWHAINHP,VANWHAIANO,VANWHASNHP,VANWHASOR,VANWHNHOPO,VANBLAIANS,VANBLAINHP,VANBLAIANO,VANBLASNHP,VANBLASOR,VANBLNHOPO,VANAIANSHP,VANAIANASO,VANAIANHPO,VANASNHPOO,VAN4RACES,VANWHBLINS,VANWHBINHP,VANWHBLINO,VANWHBLSHP,VANWHBLASO,VANWHBLHPO,VANWHINSHP,VANWHAINSO,VANWHINHPO,VANWHASHPO,VANBLINSHP,VANBLAINSO,VANBLINHPO,VANBLASHPO,VANAINSHPO,VAN5RACES,VANWBINSHP,VANWHBINSO,VANWBINHPO,VANWHBSHPO,VANWINSHPO,VANBINSHPO,VAN6RACES,VNWBINSHPO,TAHOUSING,TAHOCCUPID,TAHVACANT,TAGRPQRTR,TAINSTPOP,TAINSTCORR,TAINSTJCOR,TAINSTNURS,TAINSTOTHR,TANONINST,TAUNIVCOL,TAMILQRTR,TANINSTOTR,UFID,UID,layername,layerid,PUID,noparents,dindex,rindex,PRES202

Making subset of dataframe, limiting number of columns

In [36]:
df_subset = df[['fid', 'GEOID20', 'NAME20', 'TAPERSONS', 'PRADJPOPD', 'ADJPOP']]

In [37]:
df_subset

,fid,GEOID20,NAME20,TAPERSONS,PRADJPOPD,ADJPOP
0,1,511179308004016,Block 4016,30,1,31
1,2,511179306001060,Block 1060,15,1,16
2,3,511179304011005,Block 1005,2,0,2
3,4,510538405002007,Block 2007,0,0,0
4,5,510538401011049,Block 1049,9,0,9
...,...,...,...,...,...,...
163486,163487,510594611001008,Block 1008,38,0,38
163487,163488,510594512001007,Block 1007,39,0,39
163488,163489,510594402013008,Block 3008,0,0,0
163489,163490,510594405031003,Block 1003,17,0,17


In [38]:
# df_subset.to_csv('..//data//processed//va_districting_data_subset.csv', index = False)

What didn't work

In [33]:
# # geometry column name is in gpkg_geometry_columns
# # let'S pretend we already know it is 'Shape' for this dataset
# cursor.execute("SELECT County FROM blocks LIMIT 20;")
# result = cursor.fetchone()[0]
# # now you have the bytes of one geometry

In [23]:
# # Display data
# print('\nData in blocks table:')
# data=cursor.execute('''SELECT * FROM blocks''')
# for row in data:
#     print(row)

In [24]:
# # Commit your changes in the database
# conn.commit()

# # Closing the connection
# conn.close()

In [25]:
# # Import module
# import sqlite3

# # Connecting to sqlite
# conn = sqlite3.connect('..//data//source//RDH-data//va_pl2020_official_gpkg//statewide20.gpkg')

# # Creating a cursor object using the cursor() method
# cursor = conn.cursor()


# # Display columns
# print('\nColumns in blocks table:')
# data=cursor.execute('''SELECT * FROM blocks''')
# for column in data.description:
#     print(column[0])

# # Display data
# print('\nData in blocks table:')
# data=cursor.execute('''SELECT * FROM blocks''')
# for row in data:
#     print(row)

# # Commit your changes in the database
# conn.commit()

# # Closing the connection
# conn.close()


In [10]:
# pip install Fiona

In [11]:
# import fiona

# # No need to pass "layer='etc'" if there's only one layer
# with fiona.open('..//data//source//RDH-data//va_pl2020_official_gpkg//statewide20.gpkg', layer='layer_of_interest') as layer:
#     for feature in layer:
#         print(feature['geometry'])

In [12]:
# import geopandas as gpd

In [13]:
# data = gpd.read_file("..//data//source//RDH-data//va_pl2020_official_gpkg//statewide20.gpkg")
# data.head()  # Prints the first 5 rows of the loaded data to see what it looks like.

In [29]:
# https://stackoverflow.com/questions/18827028/write-to-csv-from-sqlite3-database-in-python

In [27]:
# import sqlite3
# import csv

# with sqlite3.connect('..//data//source//RDH-data//va_pl2020_official_gpkg//statewide20.gpkg') as connection:
#     csvWriter = csv.writer(open("output_statewide20.csv", "w"))
#     c = connection.cursor()

#     rows = c.fetchall()

#     csvWriter.writerows(rows)

In [28]:
# # geometry column name is in gpkg_geometry_columns
# # let'S pretend we already know it is 'Shape' for this dataset
# cursor.execute("SELECT Shape FROM counties LIMIT 1;")
# result = cursor.fetchone()[0]
# # now you have the bytes of one geometry